In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from collections import defaultdict
from datetime import *
import pandas as pd
import json


In [0]:
# Retrieve the parameter values
job_id = dbutils.widgets.get("job_id")
usecase_id = dbutils.widgets.get("usecase_id")

# Print the parameter values
print(f"job_id: {job_id}")
print(f"usecase_id: {usecase_id}")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-889602637696617> in <cell line: 2>()
      1 # Retrieve the parameter values
----> 2 job_id = dbutils.widgets.get("job_id")
      3 usecase_id = dbutils.widgets.get("usecase_id")
      4 
      5 # Print the parameter values

/databricks/python_shell/dbruntime/WidgetHandlerImpl.py in get(self, name)
     40         :return: Current value of the widget or default value
     41         """
---> 42         return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())
     43 
     44     def getArgument(self, name, defaultValue=None):

/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1319 
   1320         answer = self.gateway_client.send_command(command)
-> 1321         return_value = get_return_value(
   1322             answer, self.gateway_client, se

In [0]:
class GetNgeMetadata:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.metadata_json = {"Metadata":[]}    
    self.usecase_id = None
  
  def logger(self, message):
    print(message)
    
  def set_query_string(self, usecase_parm):
    query = f"""select distinct mu.usecase_id,mu.Step_Function,mo.Sequence_Nr, mo.job_id, mo.Mapping_Params from 
 com_us_alyt_ngebox.metadata_usecase mu join com_us_alyt_ngebox.metadata_usecase_job mo on mo.usecase_id = mu.usecase_id  and 
mu.usecase_id  = '{usecase_parm}'and mo.usecase_id = '{usecase_parm}'
order by mo.Sequence_Nr"""

    self.query = query
  
  def run_query_and_set_sqldf(self, usecase_id):
      self.usecase_id = usecase_id
      query_list = self.query.split(";")
      self.sql_df = []  
      for curr_query in query_list:
          if curr_query.strip() == "":
              continue
          curr_query_with_param = curr_query.replace("Usecase_parm", usecase_id)
          sql_df = sqlContext.sql(curr_query_with_param).collect()
          self.sql_df.extend(sql_df)
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_df_new['Sequence_Nr']= sql_df_new['Sequence_Nr'].astype(str)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    metadata_json = {"Metadata":sql_jsonparse} # type(metadata_json) -> dict
    data = metadata_json

    grouped_data = defaultdict(list)
    for item in data['Metadata']:
     key = (item['Step_Function'], item['usecase_id'])
     grouped_data[key].append(item)

    result = {'Metadata': []}
    for key, values in grouped_data.items():
     step_func, usecase_id = key
     group = {'Step_Function': step_func, 'usecase_id': usecase_id, 'data': values}
     result['Metadata'].append(group)
    self.metadata_json = json.dumps(result)

    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.metadata_json
      }
    }
    self.response = resp


In [0]:
usecase_id = dbutils.widgets.get("usecase_id")
nb_obj = GetNgeMetadata()
nb_obj.set_query_string(usecase_parm=usecase_id)
nb_obj.run_query_and_set_sqldf(usecase_id=usecase_id)
nb_obj.format_sqldf_to_json()
nb_obj.construct_microservice_response()
dbutils.notebook.exit(nb_obj.response)